# Recipe Recommender System

In this notebook, I expand on my previous findings in "Recipe_Analysis.ipynb" and begin the process of building a recommender system for Asian-inspired recipes from thewoksoflife.com. In this specific notebook, I will continue on basic exploratory data analysis and develop collaborative-filtering based methods to index recipes based on user input. I plan to use this data and build a full-stack web application to deploy on github allowing people to, based on specific parameters like their available time, calorie preferences, and of course choice of protein, receive a list of the top 5 recipes based on their criteria.

## Data

This CSV file contains 1450 recipes with 9 features, listed as follows:

- Average Rating    **(float64)**
- Calories          **(object)**
- Category          **(object)**
- Cook Time         **(object)**
- Ingredients       **(object)**
- Name              **(object)**
- Prep Time         **(object)**
- Review Count      **(float64)**
- Total Time        **(object)**
- URL               **(object)**

In [1]:
import pandas as pd

df = pd.read_csv("Cleaned_Recipes.csv")
display(df)

,Unnamed: 0,Name,Category,Prep Time,Cook Time,Total Time,Ingredients,Calories,Average Rating,Review Count,URL
0,0,Cantonese Chicken Feet Soup,Soup,60.0,165.0,225.0,"dried seaweed , raw shelled peanuts , c...",294.0,4.80,5.0,https://thewoksoflife.com/cantonese-chicken-fe...
1,1,Baked White Pepper Chicken Wings,Poultry,30.0,40.0,70.0,"whole chicken wings , white pepper powder ...",346.0,4.80,10.0,https://thewoksoflife.com/baked-white-pepper-c...
2,2,Restaurant-Style Chicken Lo Mein,Noodles,20.0,10.0,30.0,"boneless skinless chicken thighs , cornsta...",306.0,4.84,25.0,https://thewoksoflife.com/chicken-lo-mein/
3,3,Ginger Chicken,Poultry,10.0,20.0,30.0,"scallions , garlic , ginger , vegetabl...",206.0,5.00,6.0,https://thewoksoflife.com/ginger-chicken/
4,4,Chicken with Black Bean Sauce,Poultry,35.0,5.0,40.0,"chicken breasts , water, cornstarch, o...",293.0,4.87,15.0,https://thewoksoflife.com/chicken-black-beans/
...,...,...,...,...,...,...,...,...,...,...,...
750,1425,Beijing “Fried Sauce” Noodles – Zha Jiang Mian,Noodles,15.0,25.0,40.0,"ground pork , ¼ salt, cornstarch, ½ oil ...",446.0,4.73,11.0,https://thewoksoflife.com/beijing-fried-sauce-...
751,1429,Cantonese Chicken & Salted Fish Fried Rice,Rice,20.0,10.0,30.0,"oil, chicken breast , ½ medium onion , ...",NaN,4.84,6.0,https://thewoksoflife.com/cantonese-chicken-sa...
752,1440,Easy Mushroom Noodle Stir fry,Noodles,60.0,10.0,70.0,"dried shiitake mushrooms, ¼ onions or scal...",760.0,NaN,NaN,https://thewoksoflife.com/mushroom-noodles/
753,1449,KAITLIN’S SPECIAL NOODLES,Noodles,20.0,15.0,35.0,"fresh or dried red chilis , ½ sichuan peppe...",NaN,4.84,6.0,https://thewoksoflife.com/kaitlins-special-noo...


## Step 1: Criteria for Recommendation

Unfortunately, there are no user IDs, so it is not possible to use a collaborative filtering based model to recommend recipes based on prior users rating on different recipes. Instead, I will be utilizing the metrics "Average Rating" and "Review Count". For purposes of testing, I will be selecting the top 5 recipes based on 1) average rating, and if tied, than 2) review count.

In [16]:
time = int(input("Enter max cook time: "))
calories = int(input("Enter max calories: "))
category = input("What type of dish? Choose from Soup, Poultry, Noodles, Appetizers, Vegetarian, Rice, Bread, Beef, Seafood, Pork, Dessert: ")

def get_top_recipes(time, calories, category):
    """
    Gets top 5 recipes based on rating and review count from data that matches specified parameters.
    
    :param time: Maximum time allotted for recipe (both preparation and cook time)
    :param calories: Maximum calories desired for recipe
    :param category: Type of cuisine
    :returns: Dataframe containing all recipes that match the specified parameters
    
    """
    
    return df[(df.Category == category) & (df.Calories <= calories) & (df["Total Time"] <= time)] \
           .nlargest(5, columns = ['Average Rating', 'Review Count'])

get_top_recipes(time,calories,category)

Enter max cook time: 500
Enter max calories: 500
What type of dish? Choose from Soup, Poultry, Noodles, Appetizers, Vegetarian, Rice, Bread, Beef, Seafood, Pork, Dessert: Rice


,Unnamed: 0,Name,Category,Prep Time,Cook Time,Total Time,Ingredients,Calories,Average Rating,Review Count,URL
262,276,Crab Fried Rice,Rice,15.0,10.0,25.0,"oil, ginger , garlic , cooked white ri...",397.0,5.0,10.0,https://thewoksoflife.com/crab-fried-rice/
725,1218,Bacon and Egg Fried Rice,Rice,10.0,10.0,20.0,"oil, large eggs , lean bacon , medium ...",422.0,5.0,7.0,https://thewoksoflife.com/bacon-and-egg-fried-...
692,978,How to Make Sticky Rice (Foolproof Method!),Rice,NaN,45.0,45.0,"sweet ricesticky riceglutinous rice, water",342.0,5.0,6.0,https://thewoksoflife.com/how-to-make-sticky-r...
298,314,Pineapple Fried Rice,Rice,40.0,20.0,60.0,"shrimp , onion , carrot , ham , cook...",446.0,5.0,3.0,https://thewoksoflife.com/pineapple-fried-rice/
720,1172,Quick and Easy Coconut Rice,Rice,15.0,30.0,45.0,"jasmine rice, coconut milk, sugar, sal...",379.0,5.0,3.0,https://thewoksoflife.com/quick-easy-coconut-r...


## Step 2: Using NLP to develop recommendations based on similar ingredients

